## 타이타닉 데이터 분석
* 다양한 머신러닝 알고리즘을 이용해서   
  교차검증 방식으로 모델을 훈련시키고    
  예측 정확도를 평가해 봄

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [15]:
titanic = pd.read_csv('data/titanic.csv')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,life,seat,port
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S,live,1st,cherbourg
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S,live,1st,cherbourg
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S,dead,1st,cherbourg
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S,dead,1st,cherbourg
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S,dead,1st,cherbourg


In [5]:
titanic.info()   # 결측치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   float64
 1   survived  1309 non-null   float64
 2   name      1309 non-null   object 
 3   sex       1309 non-null   object 
 4   age       1046 non-null   float64
 5   sibsp     1309 non-null   float64
 6   parch     1309 non-null   float64
 7   ticket    1309 non-null   object 
 8   fare      1308 non-null   float64
 9   cabin     295 non-null    object 
 10  embarked  1307 non-null   object 
dtypes: float64(6), object(5)
memory usage: 112.7+ KB


In [16]:
# 레이블 분포 확인
titanic.life.value_counts()

dead    808
live    498
Name: life, dtype: int64

In [17]:
# 여러특성중 좌석분포 확인
titanic.seat.value_counts()

3rd    708
1st    321
2nd    277
Name: seat, dtype: int64

In [18]:
# 여러특성중 성별분포 확인
titanic.sex.value_counts()

male      842
female    464
Name: sex, dtype: int64

In [19]:
# 여러특성중 승선위치 분포 확인
titanic.port.value_counts()

cherbourg    1306
Name: port, dtype: int64

In [20]:
# 데이터 분석시 문자형 값보다는 숫자형 값을 더 잘 인식함
# 문자형값 -> 숫자형값을 변환하는 과정 필요

In [27]:
# 성별을 레이블인코딩으로 숫자형으로 변환 -> 파생변수

titanic['gender'] = titanic['sex'].apply(lambda x:0 if x == 'female' else 1)

titanic.iloc[:, [3, 13]].head(5)     # 3 = sex, 13 = gender

,sex,gender
0,female,0
1,male,1
2,female,0
3,male,1
4,female,0


In [30]:
# 승선위치를 레이블인코딩으로 숫자형으로 변환 -> 파생변수

titanic['harbor'] = titanic['embarked'].apply(lambda x:0 if x == 'C' else (1 if x =='S' else 2))
titanic.iloc[:, [9, 14]].head(5)

,embarked,harbor
0,S,1
1,S,1
2,S,1
3,S,1
4,S,1


In [40]:
# 분석에 필요한 컬럼을 뽑아 특성 / 레이블 생성
data = titanic.iloc[:, [0,4,5,6,8,13,14]]
target = titanic.survived

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [42]:
from sklearn.metrics import accuracy_score

In [44]:
# 훈련 / 평가 데이터 분할
Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, train_size=0.7, random_state=2111041110)

### 알고리즘 적용

In [52]:
# 의사결정나무

dtclf = DecisionTreeClassifier()
dtclf.fit(Xtrain, ytrain)
pred = dtclf.predict(Xtest)
accuracy_score(ytest, pred)

0.7678571428571429

In [50]:
# 로지스틱 회귀

lrclf = LogisticRegression()
lrclf.fit(Xtrain, ytrain)
pred = lrclf.predict(Xtest)
accuracy_score(ytest, pred)

0.8010204081632653

In [51]:
# 랜덤포레스트

rfclf = RandomForestClassifier()
rfclf.fit(Xtrain, ytrain)
pred = rfclf.predict(Xtest)
accuracy_score(ytest, pred)

0.7882653061224489

In [86]:
# 교차검증 1

dtclf = DecisionTreeClassifier()
scores = cross_val_score(dtclf, data, target, cv=10, scoring='accuracy')
np.mean(scores)

0.6950675278919554

In [75]:
lrclf = LogisticRegression(max_iter=300)
scores = cross_val_score(lrclf, data, target, cv=10, scoring='accuracy')
np.mean(scores)

0.7501820317087493

In [78]:
rfclf = RandomForestClassifier()
scores = cross_val_score(rfclf, data, target, cv=10, scoring='accuracy')
np.mean(scores)

0.7494186729301233

## 머신러닝 모델평가

In [89]:
# 정확도만으로 모델의 성능을 평가하는 것이 옳은것인가?
titanic.life.value_counts()

dead    808
live    498
Name: life, dtype: int64

In [90]:
# 성별에 따른 생존여부
titanic.groupby(['sex','life'])['life'].count()

sex     life
female  dead    127
        live    337
male    dead    681
        live    161
Name: life, dtype: int64

In [ ]:
# 여성의 생존률이 남성의 생존률보다 높기때문에
# 간단한 조건문만으로도 모델을 만들수도 있음

# => 입력값 : 여성 -> 생존
# => 입력값 : 남성 -> 사망